In [3]:
%pip install --upgrade --quiet  docx2txt langchain-community


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [55]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_spliter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 200
);
loader = Docx2txtLoader('./tax-table.docx');
document = loader.load_and_split(text_splitter=text_spliter);

In [7]:
%pip install -qU langchain-text-splitters


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [56]:
len(document)

184

In [ ]:
document

In [127]:
from dotenv import load_dotenv
load_dotenv()

from langchain_upstage import UpstageEmbeddings
embedding = UpstageEmbeddings(model="solar-embedding-1-large")  # 예시 모델명


In [221]:

import getpass
import os
import time

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from dotenv import load_dotenv
load_dotenv()

index_name = 'tax-table-index'

if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

database = PineconeVectorStore.from_documents(document, embedding, index_name=index_name)

In [262]:
query = '연봉 5천만원인 직장인 소득세는?'
retrive_docs = database.similarity_search(query=query);

In [263]:
retrive_docs

[Document(id='7cf5a7f0-2012-45dd-814b-fecd0a0b22f5', metadata={'source': './tax-table.docx'}, page_content='3. 총급여액이 7천만원 초과 1억2천만원 이하인 경우: 66만원 - [(총급여액 - 7천만원) × 1/2]. 다만, 위 금액이 50만원보다 적은 경우에는 50만원으로 한다.\n\n4. 총급여액이 1억2천만원을 초과하는 경우: 50만원 - [(총급여액 - 1억2천만원) × 1/2]. 다만, 위 금액이 20만원보다 적은 경우에는 20만원으로 한다.\n\n③ 일용근로자의 근로소득에 대해서 제134조제3항에 따른 원천징수를 하는 경우에는 해당 근로소득에 대한 산출세액의 100분의 55에 해당하는 금액을 그 산출세액에서 공제한다.<개정 2014. 1. 1.>\n\n[전문개정 2012. 1. 1.]\n\n\n\n제59조의2(자녀세액공제) ①종합소득이 있는 거주자의 기본공제대상자에 해당하는 자녀(입양자 및 위탁아동을 포함하며, 이하 이 조에서 “공제대상자녀”라 한다) 및 손자녀로서 8세 이상의 사람에 대해서는 다음 각 호의 구분에 따른 금액을 종합소득산출세액에서 공제한다. <개정 2015. 5. 13., 2017. 12. 19., 2018. 12. 31., 2019. 12. 31., 2022. 12. 31., 2023. 12. 31., 2024. 12. 31.>\n\n1. 1명인 경우: 연 25만원\n\n2. 2명인 경우: 연 55만원\n\n3. 3명 이상인 경우: 연 55만원과 2명을 초과하는 1명당 연 40만원을 합한 금액\n\n② 삭제<2017. 12. 19.>\n\n③ 해당 과세기간에 출산하거나 입양 신고한 공제대상자녀가 있는 경우 다음 각 호의 구분에 따른 금액을 종합소득산출세액에서 공제한다.<신설 2015. 5. 13., 2016. 12. 20.>\n\n1. 출산하거나 입양 신고한 공제대상자녀가 첫째인 경우: 연 30만원\n\n2. 출산하거나 입양 신고한 공제대상자녀가 

In [264]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage()

In [265]:
prompt=f"""[idntity]
- 당신은 최고의 한국 소득세법 전문가 입니다.
- [Context]를 참고해서 질문에 답변해 주세요
[Context]
{retrive_docs}

Question: {query}
"""

In [266]:
from langchain import hub
prompt= hub.pull('rlm/rag-prompt')

In [269]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [270]:
from langchain.chains import RetrievalQA

retriever =  database.as_retriever(search_kwargs={'k':5})

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt":prompt}
)



In [271]:
retriever.invoke(query)
ai_message = qa_chain({"query": query})


In [272]:
ai_message

{'query': '연봉 5천만원인 직장인 소득세는?',
 'result': '연봉 5천만원인 직장인의 소득세를 계산하려면 먼저 종합소득 과세표준을 구해야 합니다. 연봉 5천만원에서 각종 공제액을 뺀 금액이 종합소득 과세표준이 됩니다. 공제액을 뺀 후의 과세표준이 1,400만원 초과 5,000만원 이하인 경우, 세율은 15%입니다. 따라서, 소득세는 84만원 + (1,400만원 초과 금액의 15%)으로 계산됩니다. 연봉 5천만원에서 각종 공제액을 뺀 후의 과세표준이 예를 들어 3,000만원이라면, 소득세는 84만원 + (1,600만원의 15%) = 159만 6천원이 됩니다. 그러나, 근로소득세액공제를 고려해야 하므로, 정확한 소득세는 근로소득세액공제를 적용한 후 계산해야 합니다. 근로소득세액공제는 총급여액에 따라 달라지며, 연봉 5천만원인 경우 66만원 - [(총급여액 - 7천만원) × 1/2]로 계산됩니다. 연봉 5천만원인 경우 근로소득세액공제는 0원이 됩니다. 따라서, 연봉 5천만원인 직장인의 소득세는 159만 6천원으로 계산됩니다.'}

In [273]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate


dictionary = ["사람을 나타내는 표현 -> 거주자"]


prompt = ChatPromptTemplate.from_template(f"""
사용자의 질문을 보고 ,우리의 사전을 참고해서 사용자의 질문을 변경해 주세요.
 만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.       
사전 : {dictionary}
질문: {{question}}
""")

dictionary_chain = prompt | llm | StrOutputParser()

In [274]:
new_question = dictionary_chain.invoke({"question": query})

In [275]:
new_question

'질문: 연봉 5천만원인 거주자 소득세는?\n\n이 질문에서는 "직장인"이라는 표현이 사용되었지만, 제공된 사전을 참고하여 "직장인"을 "거주자"로 변경하였습니다. "직장인"은 사람을 나타내는 표현이므로, 사전에서 지정한 대체 표현으로 변경한 것입니다.'

In [276]:
tax_chain = {"query" : dictionary_chain} | qa_chain

In [ ]:
ai_response = tax_chain.invoke({"question" : query})